In [19]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

In [20]:
class PneumoniaDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        # NORMAL 폴더의 이미지 경로와 라벨 (0)
        normal_dir = os.path.join(data_dir, 'NORMAL')
        for img_name in os.listdir(normal_dir):
            self.image_paths.append(os.path.join(normal_dir, img_name))
            self.labels.append(0)

        # PNEUMONIA 폴더의 이미지 경로와 라벨 (1)
        pneumonia_dir = os.path.join(data_dir, 'PNEUMONIA')
        for img_name in os.listdir(pneumonia_dir):
            self.image_paths.append(os.path.join(pneumonia_dir, img_name))
            self.labels.append(1)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(image_path).convert('L')  # 흑백 변환

        if self.transform:
            image = self.transform(image)

        return image, label


In [23]:
# 데이터 증강 및 전처리 설정
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),      # 수평 뒤집기
    transforms.RandomRotation(10),          # 10도 이내로 회전
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # 밝기 및 대비 조절
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # 회전 없이 이동 변환만 적용
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.5], std=[0.5]) 
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 검증 데이터에는 기본 전처리만 적용
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.5], std=[0.5]) 
])


In [24]:
# 전체 훈련 데이터셋 로드 및 분할
full_dataset = PneumoniaDataset(data_dir='chest_xray/train', transform=train_transform)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# 검증 데이터는 데이터 증강을 적용하지 않기 때문에 변환을 따로 지정
val_dataset.dataset.transform = val_transform

# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)


In [25]:
# 더 깊고 복잡한 CNN 모델 정의
class EnhancedCNN(nn.Module):
    def __init__(self):
        super(EnhancedCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(128 * 28 * 28, 256)
        self.fc2 = nn.Linear(256, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = x.view(-1, 128 * 28 * 28)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# 모델 초기화 및 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EnhancedCNN().to(device)


In [26]:
# 손실 함수 정의
criterion = nn.CrossEntropyLoss()

# 옵티마이저 정의 (학습률 조정 및 옵티마이저 변경)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [27]:
# 에포크 수 설정
num_epochs = 10

# 모델 학습 및 검증 루프
for epoch in range(num_epochs):
    # 모델을 학습 모드로 설정
    model.train()
    running_loss = 0.0
    all_train_labels = []
    all_train_predictions = []

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # 옵티마이저 초기화
        optimizer.zero_grad()

        # 순전파(forward) 및 손실 계산
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 예측 결과 저장
        _, predicted = torch.max(outputs.data, 1)
        all_train_labels.extend(labels.cpu().numpy())
        all_train_predictions.extend(predicted.cpu().numpy())

        # 역전파(backward) 및 옵티마이저 스텝
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # 평균 손실 출력
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

    # 훈련 데이터 예측 지표 출력
    train_report = classification_report(all_train_labels, all_train_predictions, target_names=['NORMAL', 'PNEUMONIA'], output_dict=True)
    train_weighted_avg_recall = train_report['weighted avg']['recall']
    print(f"Training Weighted Average Recall: {train_weighted_avg_recall:.4f}")

    # 검증 모드로 설정
    model.eval()
    all_val_labels = []
    all_val_predictions = []
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # 예측 결과 저장
            all_val_labels.extend(labels.cpu().numpy())
            all_val_predictions.extend(predicted.cpu().numpy())

    # 검증 데이터 예측 지표 출력
    val_report = classification_report(all_val_labels, all_val_predictions, target_names=['NORMAL', 'PNEUMONIA'], output_dict=True)
    val_weighted_avg_recall = val_report['weighted avg']['recall']
    print(f"Validation Weighted Average Recall: {val_weighted_avg_recall:.4f}")

    # 정확도 출력
    accuracy = 100 * correct / total
    print(f'Validation Accuracy: {accuracy:.2f}%')


Epoch [1/10], Loss: 0.2116
Training Weighted Average Recall: 0.9130
Validation Weighted Average Recall: 0.9626
Validation Accuracy: 96.26%
Epoch [2/10], Loss: 0.0973
Training Weighted Average Recall: 0.9655
Validation Weighted Average Recall: 0.9751
Validation Accuracy: 97.51%
Epoch [3/10], Loss: 0.0798
Training Weighted Average Recall: 0.9684
Validation Weighted Average Recall: 0.9751
Validation Accuracy: 97.51%
Epoch [4/10], Loss: 0.0796
Training Weighted Average Recall: 0.9681
Validation Weighted Average Recall: 0.9722
Validation Accuracy: 97.22%
Epoch [5/10], Loss: 0.0527
Training Weighted Average Recall: 0.9787
Validation Weighted Average Recall: 0.9751
Validation Accuracy: 97.51%
Epoch [6/10], Loss: 0.0450
Training Weighted Average Recall: 0.9825
Validation Weighted Average Recall: 0.9741
Validation Accuracy: 97.41%
Epoch [7/10], Loss: 0.0448
Training Weighted Average Recall: 0.9832
Validation Weighted Average Recall: 0.9665
Validation Accuracy: 96.65%
Epoch [8/10], Loss: 0.0401


In [28]:
# 테스트 데이터셋 로드 (기본 전처리만 적용)
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

test_dataset = PneumoniaDataset(data_dir='chest_xray/test', transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# 모델 평가 모드로 설정
model.eval()
all_labels = []
all_predictions = []

# 테스트 데이터셋에 대한 예측 및 실제 라벨 저장
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        # 실제 라벨과 예측 라벨을 리스트에 추가
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# Classification Report 출력 및 Weighted Average Recall 추출
report = classification_report(all_labels, all_predictions, target_names=['NORMAL', 'PNEUMONIA'], output_dict=True)

# 출력 보고서에서 weighted avg recall 값 추출
weighted_avg_recall = report['weighted avg']['recall']
print("Classification Report:")
print(classification_report(all_labels, all_predictions, target_names=['NORMAL', 'PNEUMONIA']))
print(f"Weighted Average Recall: {weighted_avg_recall:.4f}")



Classification Report:
              precision    recall  f1-score   support

      NORMAL       0.98      0.40      0.57       242
   PNEUMONIA       0.73      0.99      0.84       398

    accuracy                           0.77       640
   macro avg       0.86      0.70      0.71       640
weighted avg       0.83      0.77      0.74       640

Weighted Average Recall: 0.7719


In [ ]:

# 모델 저장
torch.save(model.state_dict(), 'pneumonia_detection_model_enhanced.pth')
print("Model saved to pneumonia_detection_model_enhanced.pth")